# 📘 Apache Spark in Jupyter (Windows) — Student Guide + Practice
This single notebook includes a **clear guide** (errors & fixes) **and** runnable **practice cells**.

### What you’ll do
1) Read the short Windows setup guide (problems & solutions)
2) Run the Spark setup cell
3) Do a quick DataFrame practice (ops, aggregations)
4) Save & read Parquet
5) Stop Spark


## ✅ Quick Requirements (Windows)
- Install Java JDK (8/11/17) and set `JAVA_HOME`
- Download & extract Spark to `C:\\spark`; add `C:\\spark\\bin` to PATH
- Install Python 3 and in PowerShell/CMD run: `pip install pyspark findspark jupyter`


## 🛠 Problems & Solutions (Windows)
**1) Spark not detected** → Set `SPARK_HOME` in the notebook:
```python
import os; os.environ['SPARK_HOME'] = r'C:\\spark'
```

**2) `ModuleNotFoundError: No module named 'findspark'`** → Install:
```powershell
pip install findspark
```

**3) Wrong Python used by Spark** → Only if needed, set both:
```powershell
set PYSPARK_PYTHON=C:\path\to\python.exe
set PYSPARK_DRIVER_PYTHON=C:\path\to\python.exe
```

**4) `NameError: name 'F' is not defined`** → Import functions:
```python
from pyspark.sql import functions as F
```


In [1]:
pip install findspark

Note: you may need to restart the kernel to use updated packages.


## ▶️ 1) Spark Setup Cell (run first)

In [2]:
# ---------- Spark + Jupyter Setup (Windows) ----------
import os
# Set SPARK_HOME only if Spark is not auto-detected
os.environ['SPARK_HOME'] = r'E:\spark\spark-3.5.6-bin-hadoop3'  # change if Spark is elsewhere, or comment if already in env

# Optional: If you have multiple Python versions, point PySpark to a specific python.exe
# os.environ['PYSPARK_PYTHON'] = r'C:\\Path\\to\\python.exe'
# os.environ['PYSPARK_DRIVER_PYTHON'] = r'C:\\Path\\to\\python.exe'

try:
    import findspark
    findspark.init()
    print('✅ findspark initialized')
except ModuleNotFoundError:
    raise SystemExit('findspark not installed. In CMD/PowerShell: pip install findspark')

from pyspark.sql import SparkSession
from pyspark.sql import functions as F

spark = (SparkSession.builder
         .appName('JupyterSparkSetup')
         .config('spark.sql.shuffle.partitions','4')
         .getOrCreate())

print('✅ Spark session started:', spark.version)

✅ findspark initialized
✅ Spark session started: 3.5.6


## ▶️ 2) Sanity Check — Create a DataFrame

In [3]:
data = [
    ('Alice', 85, 12),
    ('Bob', 90, 15),
    ('Charlie', 78, 10)
]
cols = ['name','score','attendance']
df = spark.createDataFrame(data, cols)
df.show(); df.printSchema()

+-------+-----+----------+
|   name|score|attendance|
+-------+-----+----------+
|  Alice|   85|        12|
|    Bob|   90|        15|
|Charlie|   78|        10|
+-------+-----+----------+

root
 |-- name: string (nullable = true)
 |-- score: long (nullable = true)
 |-- attendance: long (nullable = true)



## ▶️ 3) Core Operations — select, filter, withColumn, orderBy

In [4]:
df2 = (df
       .select('name','score','attendance')
       .filter(F.col('attendance') > 10)
       .withColumn('score_boosted', F.col('score') + 2)
       .orderBy(F.desc('score_boosted')))
df2.show()

+-----+-----+----------+-------------+
| name|score|attendance|score_boosted|
+-----+-----+----------+-------------+
|  Bob|   90|        15|           92|
|Alice|   85|        12|           87|
+-----+-----+----------+-------------+



## ▶️ 4) Aggregations — avg, max

In [5]:
(df.groupBy()
   .agg(F.avg('score').alias('avg_score'),
        F.max('attendance').alias('max_attendance'))
   .show())

+-----------------+--------------+
|        avg_score|max_attendance|
+-----------------+--------------+
|84.33333333333333|            15|
+-----------------+--------------+



## ▶️ 5) Save & Read Back (Parquet demo)

In [6]:
out = 'students_parquet'
df.write.mode('overwrite').parquet(out)
back = spark.read.parquet(out)
back.show()

Py4JJavaError: An error occurred while calling o75.parquet.
: java.lang.RuntimeException: java.io.FileNotFoundException: java.io.FileNotFoundException: HADOOP_HOME and hadoop.home.dir are unset. -see https://wiki.apache.org/hadoop/WindowsProblems
	at org.apache.hadoop.util.Shell.getWinUtilsPath(Shell.java:735)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:270)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:286)
	at org.apache.hadoop.fs.RawLocalFileSystem.setPermission(RawLocalFileSystem.java:978)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkOneDirWithMode(RawLocalFileSystem.java:660)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirsWithOptionalPermission(RawLocalFileSystem.java:700)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirs(RawLocalFileSystem.java:672)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirsWithOptionalPermission(RawLocalFileSystem.java:699)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirs(RawLocalFileSystem.ja

## 🛑 6) Stop Spark when done

In [ ]:
spark.stop()
print('🛑 Spark session stopped.')

🛑 Spark session stopped.


## 📎 Appendix — Quick Commands
- Install packages: `pip install pyspark findspark jupyter`
- Check Python path in notebook: `!where python`
- Check Java: `java -version`
- Check Spark CLI: `pyspark --version` (from CMD where Spark bin is on PATH)
